In [1]:
# ✅ Cell 1: Setup and Configuration

import pandas as pd
from pathlib import Path

# 📂 Set local data paths (update if needed)
SNAPPED_VILLAGE_DIR = Path(r"C:/Users/utkar/OneDrive/Desktop/ClimateXTelemedicine Odisha/Odisha_VScode/.venv/Final_version/Data/snapped_villages_by_district")
SNAPPED_FACILITY_DIR = Path(r"C:/Users/utkar/OneDrive/Desktop/ClimateXTelemedicine Odisha/Odisha_VScode/.venv/Final_version/Data/snapped_facilities_by_district")
OUTPUT_BASE_DIR = Path(r"C:/Users/utkar/OneDrive/Desktop/ClimateXTelemedicine Odisha/Odisha_VScode/.venv/Final_version/Ouptuts")

# 🌍 OSRM local engine URL
OSRM_URL = "http://localhost:5000"

# 🧾 Load master file to extract district names
master_path = Path(r"C:/Users/utkar/OneDrive/Desktop/ClimateXTelemedicine Odisha/Odisha_VScode/.venv/Final_version/Data/master_dataset_final.csv")
master_df = pd.read_csv(master_path)

# ✅ Get clean district list
districts = sorted(master_df["Districtname"].dropna().str.lower().str.strip().unique())

# 🏥 Facility configuration (knn for routing strategy)
FACILITY_CONFIG = {
    "hsc": {"knn": 1, "output_subdir": "hsc"},
    "phc": {"knn": 3, "output_subdir": "phc"},
    "chc": {"knn": 3, "output_subdir": "chc"}
}

print(f"✅ Configuration loaded with {len(districts)} districts:")
print(districts)

✅ Configuration loaded with 30 districts:
['anugul', 'balangir', 'baleshwar', 'bargarh', 'bhadrak', 'boudh', 'cuttack', 'deogarh', 'dhenkanal', 'gajapati', 'ganjam', 'jagatsinghapur', 'jajapur', 'jharsuguda', 'kalahandi', 'kandhamal', 'kendrapara', 'kendujhar', 'khordha', 'koraput', 'malkangiri', 'mayurbhanj', 'nabarangpur', 'nayagarh', 'nuapada', 'puri', 'rayagada', 'sambalpur', 'sonepur', 'sundargarh']


In [4]:
# ✅ Cell 1 (Updated): Setup Paths and Configuration
import pandas as pd
from pathlib import Path

# 📂 Root folders (update if changed)
SNAPPED_VILLAGE_DIR = Path(r"C:/Users/utkar/OneDrive/Desktop/ClimateXTelemedicine Odisha/Odisha_VScode/.venv/Final_version/Data/snapped_villages_by_district")
SNAPPED_FACILITY_DIR = Path(r"C:/Users/utkar/OneDrive/Desktop/ClimateXTelemedicine Odisha/Odisha_VScode/.venv/Final_version/Data/snapped_facilities_by_district")
OUTPUT_BASE_DIR = Path(r"C:/Users/utkar/OneDrive/Desktop/ClimateXTelemedicine Odisha/Odisha_VScode/.venv/Final_version/Ouptuts")

# 🌐 OSRM Server
OSRM_URL = "http://localhost:5000"

# ✅ Load district list from master dataset
master_df = pd.read_csv(r"C:/Users/utkar/OneDrive/Desktop/ClimateXTelemedicine Odisha/Odisha_VScode/.venv/Final_version/Data/master_dataset_final.csv")
districts = sorted(master_df["Districtname"].dropna().str.lower().str.strip().unique())
print(f"✅ Configuration loaded with {len(districts)} districts.")

# ✅ Define global paths here (used in Cell 3)
villages_root = SNAPPED_VILLAGE_DIR
facilities_root = SNAPPED_FACILITY_DIR

✅ Configuration loaded with 30 districts.


In [2]:
# ✅ Cell 2: Define routing logic using OSRM and k-NN fallback

import geopandas as gpd
from shapely.geometry import Point
from sklearn.neighbors import NearestNeighbors
import requests
import numpy as np
import time
from tqdm import tqdm

def route_nearest_facility(village_gdf, facility_gdf, facility_type, district_name, osrm_url, output_dir):
    """
    Routes villages to nearest health facilities using OSRM.
    - For HSC: select nearest facility (k=1)
    - For PHC/CHC: select best of 3 closest (k=3)
    Outputs CSV with travel time + distance + identifiers.
    """
    print(f"\n🔁 Routing {facility_type.upper()} for {district_name} using k-NN + OSRM strategy...")

    k = FACILITY_CONFIG[facility_type]["knn"]
    results = []

    # ✅ Prepare coordinate arrays
    village_coords = np.array([[geom.x, geom.y] for geom in village_gdf.geometry])
    facility_coords = np.array([[geom.x, geom.y] for geom in facility_gdf.geometry])

    # ✅ Fit k-NN model
    knn = NearestNeighbors(n_neighbors=min(k, len(facility_coords)), metric='euclidean')
    knn.fit(facility_coords)
    distances, indices = knn.kneighbors(village_coords)

    for i, village in village_gdf.iterrows():
        candidates = []
        for j in indices[i]:
            fac = facility_gdf.iloc[j]
            coord_v = f"{village.geometry.x},{village.geometry.y}"
            coord_f = f"{fac.geometry.x},{fac.geometry.y}"

            try:
                # 🛣️ OSRM API call
                url = f"{osrm_url}/route/v1/driving/{coord_v};{coord_f}?overview=false"
                response = requests.get(url).json()
                if 'routes' in response and response['routes']:
                    duration = response['routes'][0]['duration'] / 60  # in minutes
                    distance = response['routes'][0]['distance'] / 1000  # in km
                    candidates.append((duration, distance, fac))
            except Exception as e:
                print(f"❌ OSRM failed for village {village['lgd_villagecode']} → facility {fac['NIN']}: {e}")
                continue

        if not candidates:
            print(f"⚠️ No valid route for village {village['lgd_villagecode']}")
            continue

        # ✅ Select best candidate: min travel time, then distance
        best = sorted(candidates, key=lambda x: (x[0], x[1]))[0]
        best_duration, best_distance, best_fac = best

        results.append({
            "lgd_villagecode": village.get("lgd_villagecode"),
            "village_name": village.get("villagename_locallang") or village.get("village_name"),
            "district": district_name,
            "NIN": best_fac["NIN"],
            "facility_name": best_fac["Facilityname"],
            "facility_type": facility_type,
            "travel_time_min": round(best_duration, 2),
            "distance_km": round(best_distance, 2)
        })

    # ✅ Save results
    out_df = pd.DataFrame(results)
    subdir = FACILITY_CONFIG[facility_type]["output_subdir"]
    out_path = output_dir / subdir / f"{district_name}_village_to_{facility_type}.csv"
    out_path.parent.mkdir(parents=True, exist_ok=True)
    out_df.to_csv(out_path, index=False)
    print(f"✅ Saved: {out_path} ({len(out_df)} records)")

In [5]:
# ✅ Cell 3: Execute routing for all 30 districts and 3 facility types

for district in tqdm(districts, desc="🏁 District Progress"):
    try:
        # 📍 Load snapped village file
        village_path = villages_root / f"{district}_villages_snapped.geojson"
        if not village_path.exists():
            print(f"❌ Village file missing: {village_path}")
            continue
        village_gdf = gpd.read_file(village_path)

        # 📍 Load snapped facility file
        facility_path = facilities_root / f"{district}_facilities_snapped.geojson"
        if not facility_path.exists():
            print(f"❌ Facility file missing: {facility_path}")
            continue
        facility_gdf = gpd.read_file(facility_path)

        # 🔁 Route for each facility type
        for facility_type in FACILITY_CONFIG.keys():
            fac_filtered = facility_gdf[
                facility_gdf["facility_type_standardized"].str.lower() == facility_type
            ].copy()

            if fac_filtered.empty:
                print(f"⚠️ No facilities of type '{facility_type}' in {district}")
                continue

            route_nearest_facility(
                village_gdf=village_gdf,
                facility_gdf=fac_filtered,
                facility_type=facility_type,
                district_name=district,
                osrm_url=OSRM_URL,
                output_dir=OUTPUT_BASE_DIR
            )

    except Exception as e:
        print(f"❌ Error processing district {district}: {e}")

🏁 District Progress:   0%|          | 0/30 [00:00<?, ?it/s]


🔁 Routing HSC for anugul using k-NN + OSRM strategy...
✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Ouptuts\hsc\anugul_village_to_hsc.csv (1889 records)

🔁 Routing PHC for anugul using k-NN + OSRM strategy...
✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Ouptuts\phc\anugul_village_to_phc.csv (1889 records)

🔁 Routing CHC for anugul using k-NN + OSRM strategy...


🏁 District Progress:   3%|▎         | 1/30 [02:40<1:17:42, 160.77s/it]

✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Ouptuts\chc\anugul_village_to_chc.csv (1889 records)

🔁 Routing HSC for balangir using k-NN + OSRM strategy...
✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Ouptuts\hsc\balangir_village_to_hsc.csv (1792 records)

🔁 Routing PHC for balangir using k-NN + OSRM strategy...
✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Ouptuts\phc\balangir_village_to_phc.csv (1792 records)

🔁 Routing CHC for balangir using k-NN + OSRM strategy...


🏁 District Progress:   7%|▋         | 2/30 [05:18<1:14:16, 159.17s/it]

...

✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Ouptuts\chc\sambalpur_village_to_chc.csv (1318 records)

🔁 Routing HSC for sonepur using k-NN + OSRM strategy...
✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Ouptuts\hsc\sonepur_village_to_hsc.csv (966 records)

🔁 Routing PHC for sonepur using k-NN + OSRM strategy...
✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Ouptuts\phc\sonepur_village_to_phc.csv (966 records)

🔁 Routing CHC for sonepur using k-NN + OSRM strategy...


🏁 District Progress:  97%|█████████▋| 29/30 [1:05:23<01:53, 113.99s/it]

✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Ouptuts\chc\sonepur_village_to_chc.csv (966 records)

🔁 Routing HSC for sundargarh using k-NN + OSRM strategy...
✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Ouptuts\hsc\sundargarh_village_to_hsc.csv (1776 records)

🔁 Routing PHC for sundargarh using k-NN + OSRM strategy...
✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Ouptuts\phc\sundargarh_village_to_phc.csv (1776 records)

🔁 Routing CHC for sundargarh using k-NN + OSRM strategy...


🏁 District Progress: 100%|██████████| 30/30 [1:07:51<00:00, 135.73s/it]

✅ Saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Ouptuts\chc\sundargarh_village_to_chc.csv (1776 records)


In [6]:
import os
import pandas as pd
from glob import glob

# === CONFIG ===
base_dir = r"C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Ouptuts"
facility_types = ["hsc", "phc", "chc"]
districts_order = sorted([
    "anugul", "balangir", "baleshwar", "bargarh", "boudh", "bhadrak",
    "cuttack", "deogarh", "dhenkanal", "gajapati", "ganjam", "jagatsinghapur",
    "jajapur", "jharsuguda", "kalahandi", "kandhamal", "kendrapara", "kendujhar",
    "khordha", "koraput", "malkangiri", "mayurbhanj", "nabarangpur", "nayagarh",
    "nuapada", "puri", "rayagada", "sambalpur", "sonepur", "sundargarh"
])

# === MERGE AND SAVE ===
for f_type in facility_types:
    print(f"🔁 Merging files for: {f_type.upper()}")
    folder = os.path.join(base_dir, f_type)
    all_files = glob(os.path.join(folder, "*_village_to_*.csv"))
    
    merged_df = pd.DataFrame()
    for district in districts_order:
        match_file = [f for f in all_files if f"{district}_village_to_{f_type}" in f]
        if match_file:
            df = pd.read_csv(match_file[0])
            df["district"] = district  # Add district column for clarity
            merged_df = pd.concat([merged_df, df], ignore_index=True)
        else:
            print(f"⚠️ Missing: {district} in {f_type}")

    # Save final merged file
    out_path = os.path.join(folder, f"master_village_to_{f_type}.csv")
    merged_df.to_csv(out_path, index=False)
    print(f"✅ Master file saved: {out_path} ({len(merged_df)} records)\n")

🔁 Merging files for: HSC
✅ Master file saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Ouptuts\hsc\master_village_to_hsc.csv (51458 records)

🔁 Merging files for: PHC
✅ Master file saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Ouptuts\phc\master_village_to_phc.csv (51464 records)

🔁 Merging files for: CHC
✅ Master file saved: C:\Users\utkar\OneDrive\Desktop\ClimateXTelemedicine Odisha\Odisha_VScode\.venv\Final_version\Ouptuts\chc\master_village_to_chc.csv (51433 records)



In [7]:
# =============================================================
# 📝 NOTE FOR RESEARCH PAPER – MISSING ROUTING OUTPUTS
# =============================================================
# The following villages did not return valid OSRM routes 
# and must be documented as exceptions in the methodology:
#
# • 🔸 Baleshwar (HSC)
#     - Missing route for village IDs: 391779.0, 391778.0
#
# • 🔸 Bhadrak (HSC)
#     - Missing route for village IDs: 394890.0, 394781.0, 
#       394776.0, 394753.0
#
# • 🔸 Rayagada (CHC)
#     - 30+ missing villages (IDs include: 427192.0 to 427158.0)
#     - These are likely remote tribal villages disconnected 
#       from the road network or with geometry errors.
#
# 🔁 These should be:
#     (1) Reported in the Data Limitations section
#     (2) Cross-validated with snapped village layers
#     (3) Flagged in maps as "no access via motorable roads"
#     (4) Used to recommend Mobile Health Unit (MHU) deployment
# =============================================================